### 黄牛检测

In [1]:
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn

In [2]:
# 读取数据
file_path = './data/raw_data.xlsx'
sheet_name = 'Sheet1'
data = pd.read_excel(file_path, sheet_name=sheet_name)

In [15]:
print(data.columns)

Index(['ID', '省份', 'APPID', 'IP_ADDRESS', '订单创建时间', '患者ID', '患者创建时间', '就诊日期',
       '就诊科室名称', '医生姓名', '状态', '商户订单号'],
      dtype='object')


#### 1. 基于规则的检测

1. IP重复，来自同一个IP，且为超过3个人挂号
2. 用户重复，来自同一个用户，且挂号了超过3个科室/超过两个app_id
3. 时间过早，每天5:00-5:01进行操作的

In [13]:
# 重复值筛选，找出data的seg_name字段中重复数大于limit的行, sort_add和asc_add是检测完之后添加的排序要求
def duplicate_detect(data, seg_name, limit, sort_add=[], asc_add=[]):
    assert len(sort_add) == len(asc_add)
    value_counts = data[seg_name].value_counts()
    dup_row = data[data[seg_name].isin(value_counts[value_counts > limit].index)]
    sort_by = [seg_name] + sort_add
    asc = [True] + asc_add
    dup_row = dup_row.sort_values(by=sort_by, ascending=asc)
    return dup_row

In [94]:
# 确定是黄牛的行编号
confirm_evil = []
IP_limit = 90
# IP重复检测
ip_dup = duplicate_detect(data, 'IP_ADDRESS', IP_limit, sort_add=['患者ID'], asc_add=[True])
print(len(ip_dup))
IP_pos = 0          # 这个IP的起点
user_count = 1      # 涉及多少个用户ID
last_user = ip_dup['患者ID'].iloc[1] # 上一个用户的ID
dup_num = len(ip_dup)
for i in range(1, dup_num):
    if ip_dup['IP_ADDRESS'].iloc[i] == ip_dup['IP_ADDRESS'].iloc[IP_pos]:
        if ip_dup['患者ID'].iloc[i] != last_user:
            # 相同IP下一个新的患者
            user_count += 1
            last_user = ip_dup['患者ID'].iloc[i]
    if ip_dup['IP_ADDRESS'].iloc[i] != ip_dup['IP_ADDRESS'].iloc[IP_pos] or i == dup_num - 1:
        # 开始检测下一个IP
        if user_count > IP_limit:
            # 达到重复人数条件
            for j in range(IP_pos, i):
                confirm_evil.append(ip_dup['ID'].iloc[j])
        # 重置
        IP_pos = i
        user_count = 1
        last_user = ip_dup['患者ID'].iloc[i]
confirm_evil.sort()
print(len(confirm_evil))

18916
8488


In [90]:
def write_list(lis):
    # 打开一个文件进行写入，如果文件不存在则创建
    with open('./data/result.txt', 'w', encoding='utf-8') as file:
        # 遍历列表中的每个元素
        for item in lis:
            # 将每个元素写入文件，每个元素后面加上换行符
            file.write(str(item) + '\n')

In [91]:
write_list(confirm_evil)